In [8]:
import pandas as pd
import numpy as np
import gensim
import gensim.downloader as gensim_api

import spacy
import xml.etree.ElementTree as ET 

In [3]:
xmlpath = '/content/drive/MyDrive/2021/stuttgart/Text Tech Team/resources/interspeech/all_formatted.xml'
tree = ET.parse(xmlpath)
root = tree.getroot() 
# https://radimrehurek.com/gensim/auto_examples/howtos/run_downloader_api.html
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [4]:
index_terms = []
for conf in root:
    meta = conf[0]
    papers = conf[1]
    
    for paper in papers:
        for key in paper[4]:
            key = key.text
            clean_key = gensim.utils.simple_preprocess(key, deacc=True)
            index_terms.append(clean_key)

In [19]:
nlp = spacy.load('en', disable=['parser', 'ner'])
def lemmatisation(words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    ret = []
    for sent in words:
        doc = nlp(" ".join(sent)) 
        ret.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return ret

# Only use open class data
lemmas = lemmatisation(index_terms)
data_uniq = list(set(lemmas))
print(len(lemmas), len(data_uniq))

32126 8367


In [35]:
def embed_index_term(terms, model):
    fail = []
    success = []
    label = []
    for keys in terms:
        keys = keys.strip().split()
        status, vec = embed_one(keys, model)
        if status:
            success.append(vec)
            label.append(' '.join(keys))
        else:
            fail.append(vec)
    return fail, success, label

def embed_one(keys, model):
    all_in = True
    for key in keys:
        if key not in model.vocab:
            all_in = False
            break
    
    if not all_in:
        return False, keys
    ret = np.zeros((50), dtype=float)
    for key in keys:
        ret += np.array(model[key], dtype=float)
    return True, ret
fail, success, label = embed_index_term(data_uniq, model)

In [40]:
# cluster 
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

kmeans = KMeans(n_clusters=100, init='k-means++')
data_mat = np.array(success)
kmeans.fit(data_mat)

KMeans(n_clusters=100)

In [41]:
import collections
cluster = collections.defaultdict(list)
for idx, c in enumerate(kmeans.labels_):
    cluster[c].append(label[idx])

In [46]:
for x in cluster[0]:
    print(x)

polyphonic sound event detection se
cross multilingual acoustic lexical modeling preprocessing
acoustic event classification detection
animated vocal model
multilingual acoustic model
acoustic feature tag perceive emphasis
acoustic event detection
acoustic unit discovery
low resource acoustic modeling
acoustic scene classification
acoustic concept index
acoustic prominence modeling
ensemble acoustic model
acoustic scene analysis
rich acoustic feature
automate acoustic analysis
acoustic model
multi dialect acoustic modeling
unsupervised acoustic unit mining
acoustic adaptation hypothesis
acoustic voice parameter
datum drive acoustic modeling
multilingual acoustic modeling
music genre classification
acoustic trading relation
acoustic model adaptation
multilingual acoustic modelling
acoustic feature
description acoustic unit
russian acoustic feature
acoustic feature compression
visual feature extraction
acoustic feature selection
acoustic methodology
acoustic background detection
robust a